<div class="header">
    <div class="title">
    <h1>Lennard Jones Potential</h1>
    </div>
    <div class="authors">
        Paul Breuer <br/>
    </div>
</div>

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import time
from ase import Atoms
from ase.visualize import view
from ase.io.trajectory import Trajectory

## Simulation parameters

In [ ]:
delta = 0.01
M = 1000
dimensions = 3
LStol = 10**-8
ECtol = 10**-10
MaxLSSteps = 10**5
MaxCGSteps = 10**5

## Global variables

In [ ]:
Time_calculating_forces = 0
goal_energy = [-1.0, -3.0, -6.0, -9.1, -12.7, -16.5, -19.8, -24.1, -28.4, -32.7, -37.9, -44.3, -47.8, -52.3, -56.8, -61.3, -66.5, -72.7, -77.2, -81.7, -86.8, -92.8, -97.3, -102.4]

## Remark:

The complete notebook might take a significant time to execute. 

# Perform energy minimization of nanoclusters

### Inputs:

- Task 1 contains:
    - initialization of the positions
    - calculate energy
    - calculate forces
    - perform line search
    - perform conjugate gradient minimization
- the simulation parameters:
    - *N*: number of atoms (2-25)
    - *L*: box size
    - *alpha*: for calculating the total energy
    - *dimensions*: number of dimensions

## Position initialization

In [ ]:
def initialize_positions(N, L):
    pos = (np.random.rand(N, dimensions) - 1/2) * L
    return pos

## Finding distance

In [ ]:
def find_distance(atom_1, atom_2, L):
    distance = atom_2 - atom_1 - L * np.round((atom_2 - atom_1) / L)
    return distance

## Potential Energy calculation

In [ ]:
def calculate_potential_energy(positions, L, epsilon=1.0, sigma=1.0, rc=2.5):
    N = len(positions)
    alpha = 0.0001 * len(positions) ** (-2 / 3)
    total_energy = 0.0
    sum_1 = 0.0
    sum_2 = 0
    for i in range(N-1):
        for j in range(i+1, N):
            r_ij = find_distance(positions[i], positions[j], L)
            r = np.linalg.norm(r_ij)

            # Truncate the pairwise potential
            if rc * sigma >= r > 0.1:
                # Lennard-Jones potential energy calculation
                potential_energy = 4 * epsilon * ((((sigma/r)**3)**2)**2 - ((sigma/r)**3)**2 - (((sigma/rc)**3)**2)**2 + ((sigma/rc)**3)**2)
                sum_2 += potential_energy
    for i in range(N):
        sum_1 += alpha * (np.linalg.norm(positions[i])) ** 2
    energy = sum_1 + sum_2

    return energy

## Forces calculation

In [ ]:
def calculate_forces(positions, L, epsilon=1, sigma=1, rc=2.5):
    f = [[0., 0., 0.] for l in range(len(positions))]
    for i in range(len(positions) + 1):
        for j in range(i + 1, len(positions)):
            R = find_distance(positions[i], positions[j], L)
            r = np.linalg.norm(R)
            if r <= rc:
                f[i] += R / r * 4 * epsilon * (-12 * (r / sigma) ** (-13) + 6 * (r / sigma) ** (-7)) / sigma
                f[j] -= R / r * 4 * epsilon * (-12 * (r / sigma) ** (-13) + 6 * (r / sigma) ** (-7)) / sigma

    for i in range(len(f)):
        f[i] -= 2 * 0.0001 * len(positions) ** (-2 / 3) * (positions[i])
    f = np.clip(f, -(10**4), 10**4)
    return f

## Combination of forces and potential energy calculation

In [1]:
def calculate_energy_and_forces(positions, L, epsilon=1.0, sigma=1.0, rc=2.5):
    N = len(positions)
    alpha = 0.0001 * len(positions) ** (-2 / 3)
    total_energy = 0.0
    sum_1 = 0.0
    sum_2 = 0

    # energy calculation
    for i in range(N - 1):
        for j in range(i + 1, N):
            r_ij = find_distance(positions[i], positions[j], L)
            r = np.linalg.norm(r_ij)

            # Truncate the pairwise potential
            if rc * sigma >= r > 0.1:
                # Lennard-Jones potential energy calculation
                potential_energy = 4 * epsilon * ((((sigma / r) ** 3) ** 2) ** 2 - ((sigma / r) ** 3) ** 2 - (
                            ((sigma / rc) ** 3) ** 2) ** 2 + ((sigma / rc) ** 3) ** 2)
                sum_2 += potential_energy
    for i in range(N):
        sum_1 += alpha * (np.linalg.norm(positions[i])) ** 2
    energy = sum_1 + sum_2

    # force calculation
    f = [[0., 0., 0.] for l in range(len(positions))]
    for i in range(len(positions) + 1):
        for j in range(i + 1, len(positions)):
            R = find_distance(positions[i], positions[j], L)
            r = np.linalg.norm(R)
            if r <= rc:
                f[i] += R / r * 4 * epsilon * (-12 * (r / sigma) ** (-13) + 6 * (r / sigma) ** (-7)) / sigma
                f[j] -= R / r * 4 * epsilon * (-12 * (r / sigma) ** (-13) + 6 * (r / sigma) ** (-7)) / sigma

    for i in range(len(f)):
        f[i] -= 2 * 0.0001 * len(positions) ** (-2 / 3) * (positions[i])
    f = np.clip(f, -(10 ** 4), 10 ** 4)

    return energy, f

## Quadratic interpolation

In [ ]:
def interpolation(energy_0, energy_1, energy_2, d):
    a = np.zeros(3)
    x = np.array([0, d, d * 2])
    y = np.array([energy_0, energy_1, energy_2])

    a = [[0., 0., 0.] for i in range(3)]
    for i in range(3):
        a[i] = np.array([x[i] ** 2, x[i], 1.])
    poly = np.linalg.solve(a, y)
    delta_min = -poly[1] / (2 * poly[0])

    return delta_min

## Line Search

In [ ]:
def line_search(positions, d, L, delta_initial=0.001, LStol=10e-8, MaxLSSteps=10**5):
    i = 0
    N = len(positions)
    delta = delta_initial
    final_positions = positions
    final_potential_energy = calculate_potential_energy(final_positions, L)
    r0 = positions
    r1 = positions
    r2 = positions
    for j in range(len(positions)):
        r1[j] = r0[j] + delta * d[j]
    for j in range(len(positions)):
        r2[j] = r1[j] + delta * d[j]
    U_r0 = calculate_potential_energy(r0, L)
    U_r1 = calculate_potential_energy(r1, L)
    U_r2 = calculate_potential_energy(r2, L)
    while (i < MaxLSSteps) and not (U_r0 > U_r1 and U_r2 > U_r1):
        i += 1
        r1 = r2
        for j in range(N):
            r2[j] = r1[j] + delta * d[j]
        U_r0 = U_r1
        U_r1 = U_r2
        U_r2 = calculate_potential_energy(r2, L)
        fractional_change = np.abs((U_r1 - U_r2) / U_r1)
        delta *= 1.1 if fractional_change < 0.9 else 0.5
        if fractional_change < LStol:
            break
    if (U_r2 > U_r1) and (U_r0 > U_r1):
        delta_min = interpolation(U_r0, U_r1, U_r2, delta)
        for j in range(len(positions)):
            final_positions[j] = r0[j] + delta_min * d[j]
        final_potential_energy = calculate_potential_energy(final_positions, L)
    return final_positions, final_potential_energy

## Conjugate Gradient minimization

In [ ]:
def conjugate_gradient_minimization(positions, L, ECtol=1e-10, MaxCGSteps=100000):
    energy = np.empty(MaxCGSteps + 1)
    energy[:] = 0.
    CGSteps = 0
    old_energy, direction = calculate_energy_and_forces(positions, L)
    energy[CGSteps] = old_energy
    if np.linalg.norm(direction) < 0.0001:
        return positions, old_energy, CGSteps, energy

    old_forces = direction
    direction = np.array([d / np.linalg.norm(d) for d in direction])
    old_energy = calculate_potential_energy(positions, L)
    r_0, new_energy = line_search(positions, direction, L)
    diff = abs(new_energy - old_energy)
    CGSteps += 1
    energy[CGSteps] = new_energy
    while diff >= ECtol * abs(new_energy) and CGSteps < MaxCGSteps:
        r_copy = np.array(r_0)
        new_forces = calculate_forces(r_0, L)
        if np.linalg.norm(new_forces) < 0.0001:
            return r_0, new_energy, CGSteps, energy

        y = [(x - old_forces[i]).dot(x) / old_forces[i].dot(old_forces[i]) for i, x in enumerate(new_forces)]
        direction = np.array([new_forces[i] + x * direction[i] for i, x in enumerate(y)])
        direction = np.array([d / np.linalg.norm(d) for d in direction])
        old_energy = new_energy
        old_forces = new_forces
        r_0, new_energy = line_search(r_0, direction, L)
        diff = abs(new_energy - old_energy)
        if (new_energy > 0 > old_energy) or (0.8 * old_energy < new_energy < 0):
            return r_copy, old_energy, CGSteps, energy
        CGSteps += 1
        energy[CGSteps] = new_energy

    return r_0, new_energy, CGSteps, energy

# Minimization for different starting positions and Plotting

In [ ]:
def perform_the_minimization_for_different_starting_positions(N_divided_by_V):
    N_values = np.arange(2, 26)
    L_values = np.power(N_values / N_divided_by_V, 1 / 3)
    min_CGSteps = np.zeros([N_values.size])
    min_energy = np.full([N_values.size], 1000, dtype=float)
    best_position_sequences = np.zeros([N_values.size, MaxLSSteps, 2, 3])
    best_internal_energy_sequences = np.full([N_values.size], 1000)
    best_starting_positions = np.empty(N_values.size, dtype=object)
    e_min = np.zeros(MaxCGSteps + 1)
    CGSteps_min = np.zeros(N_values.size)
    for j in range(N_values.size):
        i = 0
        for i in range(M):
            print(i)
            starting_positions = initialize_positions(N_values[j], L_values[j])
            starting_positions, energy, CGSteps, e = conjugate_gradient_minimization(starting_positions, L_values[j], ECtol, MaxCGSteps)
            if energy < min_energy[j]:
                min_energy[j] = energy
                best_starting_positions[j] = starting_positions
                CGSteps_min[j] = int(CGSteps)
                e_min[0:int(CGSteps_min[j])] = np.copy(e[0:int(CGSteps_min[j])])
            print(min_energy)
        for i in range(int(CGSteps_min[j])):
            if e_min[i] > 10:
                e_min[i] = 10

        # Printing results
        print('Results for N: ', N_values[j])
        print('Needed CGSteps: ', CGSteps_min[j])
        print('Minimum Energy: ', min_energy[j])
        print('Optimal Energy (Paper): ', goal_energy[N_values[j] - 2])
        print('------------------------------------')

        # Plotting of Task 3
        fig = plt.figure()
        ax = fig.add_axes([0, 0, 1, 1])
        ax.set_title("Potential energy at each iteration for a cluster of " + str(N_values) + " atoms")
        ax.set_ylabel("Potential Energy")
        ax.set_xlabel("Iteration")
        ax.plot(e_min[0:int(CGSteps_min[j])], linestyle='-', marker='o')
        lower = int(round(CGSteps_min[j] / 6.))
        if lower == 0:
            lower = 1
        ax.set_xticks(range(0, int(CGSteps_min[j]), lower))
        plt.show()

    traj = Trajectory('nano.traj', 'w')
    fac = 1.0
    for i in range(0, N_values.size):
        n = N_values[i]
        if best_starting_positions[i][0][0] == 0:
            break
        else:
            nano = Atoms('C' + str(n), best_starting_positions[i] * fac)
            traj.write(nano)

    # Plotting of Task 2
    plot_results_task2(N_values, CGSteps_min, min_energy)

    # Plotting of Task 4
    parameters, covariance = curve_fit(U_macro, N_values, min_energy)
    fit_y = U_macro(N_values, parameters[0], parameters[1], parameters[2])
    ax = plt.axes()
    ax.plot(N_values, min_energy - fit_y)
    ax.set_xlabel("Atoms")
    ax.set_ylabel("U/e")
    ax.set_xticks(range(N_values[0], N_values[-1], 2))
    ax.set_title("Potential energy simulation minus estimation")
    plt.show()

## Plot Results for Task 2

In [ ]:
def plot_results_task2(N, CGSteps, energy_min):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.plot(N, energy_min, marker='o')
    plt.title('Lennard-Jones Potential Minimum')
    plt.xlabel('Number of Particles (N)')
    plt.ylabel('Lennard-Jones Potential (U_min)')
    plt.ylim(-110, 10)

    plt.subplot(1, 2, 2)
    plt.plot(N, CGSteps, marker='o', color='orange')
    plt.title('Conjugate Gradient Steps')
    plt.xlabel('Number of Particles (N)')
    plt.ylabel('CG Steps to Convergence')

    plt.tight_layout()
    plt.show()

## Function for macroscopic scaling estimate

In [ ]:
def U_macro(N, a, b, c):
    return a + b * N**(2/3) + c * N

# Main

In [ ]:
if __name__ == '__main__':
    t_0 = time.perf_counter()

    # Task 1
    perform_the_minimization_for_different_starting_positions(0.01)

    # Bonus Task
    perform_the_minimization_for_different_starting_positions(0.005)

    t_end = time.perf_counter()
    print("total time elapsed:", np.round(t_end - t_0, 3), " seconds")

## Question: Do you notice anything different at certain numbers?

From the previous lectures we know that for 13 and 19 atoms we get a icosahedral cluster, where there are lower energies than would be expected according to macroscopic scaling.
Due to the fact that our energies only resemble the values of the paper up to N=12, the phenomenon is unfortunately not recognizable in our plots.